In [2]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#df short for data frame
train_bodies = 'train_bodies.csv'
bodies_df = pd.read_csv(train_bodies)
print(bodies_df.head(5))

train_stances = 'train_stances.csv'
stances_df = pd.read_csv(train_stances)
print(stances_df.head(5))

stances_df.columns = [c.replace(' ', '_') for c in stances_df.columns]
bodies_df.columns = [c.replace(' ', '_') for c in bodies_df.columns]

print
print(len(stances_df.Body_ID.unique()))
print(len(bodies_df))
print(len(stances_df))

   Body ID                                        articleBody
0        0  A small meteorite crashed into a wooded area i...
1        4  Last week we hinted at what was to come as Ebo...
2        5  (NEWSER) – Wonder how long a Quarter Pounder w...
3        6  Posting photos of a gun-toting child online, I...
4        7  At least 25 suspected Boko Haram insurgents we...
                                            Headline  Body ID     Stance
0  Police find mass graves with at least '15 bodi...      712  unrelated
1  Hundreds of Palestinians flee floods in Gaza a...      158      agree
2  Christian Bale passes on role of Steve Jobs, a...      137  unrelated
3  HBO and Apple in Talks for $15/Month Apple TV ...     1034  unrelated
4  Spider burrowed through tourist's stomach and ...     1923   disagree

1683
1683
49972


In [4]:
stances_df["Body_Text"] = ""
print(stances_df.head(10))

                                            Headline  Body_ID     Stance  \
0  Police find mass graves with at least '15 bodi...      712  unrelated   
1  Hundreds of Palestinians flee floods in Gaza a...      158      agree   
2  Christian Bale passes on role of Steve Jobs, a...      137  unrelated   
3  HBO and Apple in Talks for $15/Month Apple TV ...     1034  unrelated   
4  Spider burrowed through tourist's stomach and ...     1923   disagree   
5  'Nasa Confirms Earth Will Experience 6 Days of...      154      agree   
6  Accused Boston Marathon Bomber Severely Injure...      962  unrelated   
7  Identity of ISIS terrorist known as 'Jihadi Jo...     2033  unrelated   
8  Banksy 'Arrested & Real Identity Revealed' Is ...     1739      agree   
9      British Aid Worker Confirmed Murdered By ISIS      882  unrelated   

  Body_Text  
0            
1            
2            
3            
4            
5            
6            
7            
8            
9            


In [5]:
#Add the appropriate body text to each row (using the body id)
for index, i in enumerate(stances_df.itertuples()):
    for j in bodies_df.itertuples():
        if i.Body_ID == j.Body_ID:
            stances_df.at[index,'Body_Text'] = j.articleBody

In [6]:
#Drop Body_ID column, rearrange columns
stances_df = stances_df[['Headline','Body_Text','Stance']]
print(stances_df)

                                                Headline  \
0      Police find mass graves with at least '15 bodi...   
1      Hundreds of Palestinians flee floods in Gaza a...   
2      Christian Bale passes on role of Steve Jobs, a...   
3      HBO and Apple in Talks for $15/Month Apple TV ...   
4      Spider burrowed through tourist's stomach and ...   
5      'Nasa Confirms Earth Will Experience 6 Days of...   
6      Accused Boston Marathon Bomber Severely Injure...   
7      Identity of ISIS terrorist known as 'Jihadi Jo...   
8      Banksy 'Arrested & Real Identity Revealed' Is ...   
9          British Aid Worker Confirmed Murdered By ISIS   
10                                        Gateway Pundit   
11     Woman detained in Lebanon is not al-Baghdadi's...   
12     Kidnapped Nigerian schoolgirls: Government cla...   
13     No, that high school kid didn't make $72 milli...   
14     Soon Marijuana May Lead to Ticket, Not Arrest,...   
15     Vandals add rude paint job to $2.

In [25]:
#Apply Scikit Learn TFIDF Feature Extraction Algorithm
from sklearn.feature_extraction.text import TfidfVectorizer
body_vectorizer = TfidfVectorizer(ngram_range=(1, 3), lowercase=True, stop_words='english')
X_body = body_vectorizer.fit_transform(stances_df['Body_Text'])

headline_vectorizer = TfidfVectorizer(ngram_range=(1, 3), lowercase=True, stop_words='english')
X_headline = headline_vect.fit_transform(stances_df['Headline'])

In [26]:
X_body[0]

<1x482271 sparse matrix of type '<type 'numpy.float64'>'
	with 281 stored elements in Compressed Sparse Row format>

In [27]:
X_headline[0]

<1x19830 sparse matrix of type '<type 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [28]:
print(X_body)

  (0, 111622)	0.07332486904279714
  (0, 61926)	0.11062358813848425
  (0, 125574)	0.09436328724448594
  (0, 448715)	0.09620281606126824
  (0, 158554)	0.06121278707581918
  (0, 381761)	0.039478725956742046
  (0, 359171)	0.08017103385429222
  (0, 149291)	0.051972696403530645
  (0, 315341)	0.09230802764085554
  (0, 33684)	0.07507013612897592
  (0, 166231)	0.03303528239637393
  (0, 407023)	0.06514224536738578
  (0, 476203)	0.08019141018236453
  (0, 395340)	0.038881204072746076
  (0, 221486)	0.1692701173979176
  (0, 56006)	0.03695559013641697
  (0, 49547)	0.02305411365934867
  (0, 460278)	0.03851537680773665
  (0, 215190)	0.03917419126394295
  (0, 59820)	0.038118581066324334
  (0, 17420)	0.05709566097575283
  (0, 11454)	0.03592709280337572
  (0, 395698)	0.036095354657982366
  (0, 31411)	0.039964295283783646
  (0, 56077)	0.05709566097575283
  :	:
  (49971, 297508)	0.06264392079607924
  (49971, 262343)	0.06264392079607924
  (49971, 328442)	0.06264392079607924
  (49971, 317155)	0.06264392079607

In [29]:
print(X_headline)

  (0, 13136)	0.25043034698296257
  (0, 10770)	0.11296580149690795
  (0, 7240)	0.12974737699179834
  (0, 136)	0.14143090080058232
  (0, 2345)	0.1411928200804065
  (0, 11679)	0.12092403452543342
  (0, 11036)	0.10796696315375601
  (0, 17862)	0.15393153255284525
  (0, 367)	0.14410585941849302
  (0, 16948)	0.11426349332832958
  (0, 5131)	0.18009401018412563
  (0, 3791)	0.18009401018412563
  (0, 13151)	0.1676512469521798
  (0, 10779)	0.1351518585072518
  (0, 7241)	0.18009401018412563
  (0, 139)	0.18009401018412563
  (0, 2353)	0.18009401018412563
  (0, 11690)	0.18009401018412563
  (0, 11059)	0.18009401018412563
  (0, 17863)	0.18009401018412563
  (0, 370)	0.15503582124283458
  (0, 16959)	0.18009401018412563
  (0, 5132)	0.18009401018412563
  (0, 13137)	0.18009401018412563
  (0, 13153)	0.18009401018412563
  :	:
  (49970, 2446)	0.25226273309179803
  (49970, 9898)	0.25226273309179803
  (49970, 8662)	0.23233767736407243
  (49970, 6334)	0.25226273309179803
  (49970, 1400)	0.25226273309179803
  (4997

In [31]:
from scipy.sparse import hstack
X = hstack([body_text_vect_tfidf, headline_vect_tfidf])

In [32]:
print(X)

  (0, 111622)	0.07332486904279714
  (0, 61926)	0.11062358813848425
  (0, 125574)	0.09436328724448594
  (0, 448715)	0.09620281606126824
  (0, 158554)	0.06121278707581918
  (0, 381761)	0.039478725956742046
  (0, 359171)	0.08017103385429222
  (0, 149291)	0.051972696403530645
  (0, 315341)	0.09230802764085554
  (0, 33684)	0.07507013612897592
  (0, 166231)	0.03303528239637393
  (0, 407023)	0.06514224536738578
  (0, 476203)	0.08019141018236453
  (0, 395340)	0.038881204072746076
  (0, 221486)	0.1692701173979176
  (0, 56006)	0.03695559013641697
  (0, 49547)	0.02305411365934867
  (0, 460278)	0.03851537680773665
  (0, 215190)	0.03917419126394295
  (0, 59820)	0.038118581066324334
  (0, 17420)	0.05709566097575283
  (0, 11454)	0.03592709280337572
  (0, 395698)	0.036095354657982366
  (0, 31411)	0.039964295283783646
  (0, 56077)	0.05709566097575283
  :	:
  (49970, 484717)	0.25226273309179803
  (49970, 492169)	0.25226273309179803
  (49970, 490933)	0.23233767736407243
  (49970, 488605)	0.25226273309179

In [34]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)
Y = list(stances_df['Stance'])

In [43]:
y_pred = nb_classifier.fit(X, Y).predict(X)
print(list(y_pred))

['discuss', 'agree', 'unrelated', 'unrelated', 'disagree', 'agree', 'unrelated', 'unrelated', 'agree', 'unrelated', 'discuss', 'disagree', 'unrelated', 'disagree', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'discuss', 'disagree', 'discuss', 'agree', 'agree', 'agree', 'disagree', 'unrelated', 'agree', 'unrelated', 'discuss', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'discuss', 'agree', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'disagree', 'unrelated', 'disagree', 'agree', 'agree', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'disagree', 'discuss', 'agree', 'discuss', 'unrelated', 'unrelated', 'unrelated', 'disagree', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'disagree', 'unrelated', 'unrelated', 'unrelated', 'unrelated', 'agree', 'unrelated', 'unrelated', 'discuss', 'unrelated', 'discuss', 'unrelated', 'unrelated', 'agree', 'unrelated', 'unrelated', 'unrelated',

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(Y, y_pred)

0.7181221484031057

In [45]:
stances_df['NB_Predicted_Stance'] = list(y_pred)

In [46]:
stances_df

,Headline,Body_Text,Stance,NB_Predicted_Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,unrelated,discuss
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,agree,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,unrelated,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,unrelated,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",disagree,disagree
5,'Nasa Confirms Earth Will Experience 6 Days of...,Thousands of people have been duped by a fake ...,agree,agree
6,Accused Boston Marathon Bomber Severely Injure...,A British fighter who travelled to Iraq to sto...,unrelated,unrelated
7,Identity of ISIS terrorist known as 'Jihadi Jo...,"Adding to Apple's iOS 8 launch troubles, a rep...",unrelated,unrelated
8,Banksy 'Arrested & Real Identity Revealed' Is ...,If you’ve seen a story floating around on your...,agree,agree
9,British Aid Worker Confirmed Murdered By ISIS,The British Islamic State militant who has fea...,unrelated,unrelated
